# 🚀 Ailo Colab Miner (AILO-1B)

**Mine AiloCoin with Google Colab's free T4 GPU!**

1. Runtime > Change runtime type > T4 GPU
2. Enter wallet below
3. Runtime > Run all

✨ **v1.6.1**: Fixed gradient sync + checkpoint download!

In [ ]:
#@title ⚙️ 1. Setup
!pip install torch --index-url https://download.pytorch.org/whl/cu118 -q
!pip install aiohttp requests -q
import gc; gc.collect()
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)} | VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")
else:
    print("❌ Enable GPU: Runtime > Change runtime type > T4")

In [ ]:
#@title 🔑 2. Wallet
WALLET = ""  #@param {type:"string"}
print(f"✅ {WALLET[:12]}..." if len(WALLET)>=40 else "❌ Enter wallet from https://ailo.site/wallet.html")

In [ ]:
#@title 🧠 3. AILO-1B Model + Checkpoint Sync
import torch, torch.nn as nn, numpy as np, requests, aiohttp, asyncio, time, base64, gzip, gc, os

API="https://ailo.site/api"
SERVER="https://ailo.site"
VER="1.6.1-colab"

class AILO1B(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(50257,1600)
        self.pos=nn.Parameter(torch.zeros(1,512,1600))
        self.tf=nn.TransformerEncoder(nn.TransformerEncoderLayer(1600,25,6400,0.1,batch_first=True),24)
        self.out=nn.Linear(1600,50257)
    def forward(self,x):
        return self.out(self.tf(self.emb(x)*40+self.pos[:,:x.size(1)]))

class Trainer:
    def __init__(self):
        self.dev=torch.device('cuda')
        self.m=None; self.opt=None; self.steps=0; self.acc=0
        self.best_loss=float('inf'); self.local_version=0
    
    def init(self):
        print("🧠 Loading AILO-1B...")
        gc.collect(); torch.cuda.empty_cache()
        self.m=AILO1B().half().to(self.dev)
        print(f"   {sum(p.numel() for p in self.m.parameters()):,} params | GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB")
        self.opt=torch.optim.SGD(self.m.parameters(),lr=0.001,momentum=0.9)
        self.crit=nn.CrossEntropyLoss()
        print("   ✅ Ready!")
    
    def sync_with_global(self):
        """Download trained checkpoint or gradients from network."""
        try:
            print("🔄 Checking for global model...")
            r=requests.get(f"{SERVER}/api/model/weights",timeout=30)
            if r.status_code!=200:
                print("   No global model yet")
                return False
            
            data=r.json()
            
            # Try checkpoint first
            if data.get('hasCheckpoint',False):
                print(f"   📥 Downloading checkpoint (loss: {data.get('bestCheckpointLoss','N/A')})...")
                cr=requests.get(f"{SERVER}/api/model/checkpoint",timeout=60)
                if cr.status_code==200:
                    self._load_checkpoint(cr.content)
                    print(f"   ✅ Loaded checkpoint from {data.get('checkpointWallet','unknown')}")
                    return True
            
            # Fallback to gradients
            sv=data.get('version',0)
            if sv<=self.local_version:
                print(f"   Already at v{self.local_version}")
                return True
            
            if data.get('downloadUrls',{}).get('gradients'):
                print(f"   📥 Downloading gradients v{sv}...")
                gr=requests.get(f"{SERVER}{data['downloadUrls']['gradients']}",timeout=60)
                if gr.status_code==200:
                    self._apply_gradients(gr.content)
                    self.local_version=sv
                    print(f"   ✅ Synced to v{sv}")
                    return True
            return False
        except Exception as e:
            print(f"   ⚠️ Sync failed: {e}")
            return False
    
    def _load_checkpoint(self, data):
        """Load weights into fc_out layer."""
        try:
            w=np.frombuffer(data,dtype=np.float16).astype(np.float32)
            print(f"   Received {len(w):,} weights")
            with torch.no_grad():
                exp=self.m.out.weight.numel()
                if len(w)>=exp:
                    t=torch.from_numpy(w[:exp]).reshape_as(self.m.out.weight)
                    self.m.out.weight.data=t.half().to(self.dev)
                    print(f"   ✅ Loaded {exp:,} weights into fc_out")
        except Exception as e:
            print(f"   ⚠️ Load failed: {e}")
    
    def _apply_gradients(self, data):
        """Apply gradients to fc_out (handles sampled 1:10)."""
        try:
            g=np.frombuffer(data,dtype=np.float16).astype(np.float32)
            print(f"   Received {len(g):,} gradient values")
            with torch.no_grad():
                out=self.m.out
                full=out.weight.numel()
                sampled=full//10
                lr=0.0001
                if len(g)>=full:
                    t=torch.from_numpy(g[:full]).reshape_as(out.weight).half().to(self.dev)
                    out.weight.data-=lr*t
                    print(f"   ✅ Applied full gradients")
                elif len(g)>=sampled*0.9:
                    # Sampled 1:10 - apply to every 10th
                    flat=out.weight.data.flatten()
                    gt=torch.from_numpy(g).to(flat.device)
                    for i in range(min(len(g),len(flat)//10)):
                        flat[i*10]-=lr*gt[i]
                    out.weight.data=flat.reshape_as(out.weight)
                    print(f"   ✅ Applied sampled gradients to {len(g):,} positions")
        except Exception as e:
            print(f"   ⚠️ Apply failed: {e}")
    
    def upload_checkpoint(self, wallet, loss, epoch):
        """Upload our weights if we have good loss."""
        try:
            with torch.no_grad():
                w=self.m.out.weight.data.cpu().half().numpy().tobytes()
                w64=base64.b64encode(w).decode()
            r=requests.post(f"{SERVER}/api/model/checkpoint",json={
                'wallet':wallet,'weights':w64,'loss':loss,'epoch':epoch
            },timeout=60)
            if r.status_code==200 and r.json().get('success'):
                print(f"   📦 Checkpoint uploaded (loss: {loss:.4f})")
                return True
        except:pass
        return False
    
    def batch(self,w):
        try:
            r=requests.get(f"{API}/cuda/training-data",params={'batchSize':1,'wallet':w},timeout=10)
            if r.ok:
                t=r.json().get('articles',[''])[0]
                tk=[ord(c)%50257 for c in t[:33]]
                tk+=[0]*(33-len(tk))
                return torch.tensor([tk[:32]]),torch.tensor([tk[1:33]])
        except:pass
        return torch.randint(0,50257,(1,32)),torch.randint(0,50257,(1,32))
    
    def step(self,x,y):
        self.m.train()
        x,y=x.to(self.dev),y.to(self.dev)
        with torch.cuda.amp.autocast():
            loss=self.crit(self.m(x).view(-1,50257),y.view(-1))/16
        loss.backward()
        self.acc+=1
        if self.acc>=16:
            nn.utils.clip_grad_norm_(self.m.parameters(),1.0)
            self.opt.step(); self.opt.zero_grad(set_to_none=True); self.acc=0
        self.steps+=1
        l=loss.item()*16
        if l<self.best_loss: self.best_loss=l
        if self.steps%50==0: gc.collect(); torch.cuda.empty_cache()
        return l
    
    def grads(self):
        g=self.m.out.weight.grad
        if g is None: return None
        gc.collect()
        flat=g.cpu().float().flatten()
        sample=flat[::10]
        del flat; gc.collect()
        comp=gzip.compress(sample.half().numpy().tobytes(),4)
        del sample; gc.collect()
        return base64.b64encode(comp).decode()

print("✅ Ready (v1.6.1 with fixed gradient sync)")

In [ ]:
#@title 🚀 4. Mine!

async def hb(w,tps):
    try:
        async with aiohttp.ClientSession() as s:
            await s.post(f"{API}/cuda/register",json={'wallet':w,'clientVersion':VER,'deviceInfo':{'gpu_name':'T4-Colab','vram_gb':15,'hashrate':tps}})
            await s.post(f"{API}/ping",json={'wallet':w,'status':'training','hashrate':f"{tps:.0f} Tok/s"})
    except:pass

async def submit(w,g,s,l):
    try:
        async with aiohttp.ClientSession() as ss:
            async with ss.post(f"{API}/cuda/submit",json={'wallet':w,'gradients':g,'epoch':s,'loss':l,'gpu':'T4-Colab'}) as r:
                if r.status==200: return (await r.json()).get('reward',0)
    except:pass
    return 0

async def run():
    if len(WALLET)<40: print("❌ Set wallet!"); return
    print("="*50)
    print(f"  AILO-1B Colab Miner v{VER}")
    print("  🔄 Auto-syncs with trained network model!")
    print("="*50)
    
    await hb(WALLET,0)
    t=Trainer(); t.init()
    
    # Download checkpoint/gradients from network
    t.sync_with_global()
    
    print("\n⛏️ MINING!")
    print("📊 Dashboard: https://ailo.site/dashboard.html\n")
    
    rew,last_s,last_h,tps=0.0,time.time(),time.time(),0.0
    
    try:
        while True:
            t0=time.time()
            x,y=t.batch(WALLET)
            loss=t.step(x,y)
            tps=32/(time.time()-t0+0.001)
            
            if time.time()-last_h>=10: await hb(WALLET,tps); last_h=time.time()
            if t.steps%25==0: print(f"Step {t.steps} | Loss: {loss:.4f} | Best: {t.best_loss:.4f} | v{t.local_version} | {tps:.0f} tok/s")
            
            if time.time()-last_s>=300:
                print("\n📤 Submitting gradients...")
                gc.collect(); torch.cuda.empty_cache()
                g=t.grads()
                if g:
                    r=await submit(WALLET,g,t.steps,loss); rew+=r
                    print(f"💰 +{r:.4f} ALC (Total: {rew:.4f})")
                    if loss<4.0: t.upload_checkpoint(WALLET,loss,t.steps)
                
                # Sync every 10 min
                t.sync_with_global()
                last_s=time.time()
                print()
                
    except KeyboardInterrupt: print(f"\n⏹️ Total: {rew:.4f} ALC")

await run()